In [1]:
import pandas as pd
import json

import psycopg2
conn = psycopg2.connect(host='ec2-107-20-166-28.compute-1.amazonaws.com',
                        user='rbhciitblovwew',
                        database='db274el0c49tdu')

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(rc={"figure.figsize": (16, 9)})
sns.set(style="whitegrid")

In [2]:
query = """
WITH step_data AS (
    SELECT
    user_id
    , (data -> 'activities-steps' ->> 0)::json ->> 'dateTime' as step_day
    , json_array_elements(data -> 'activities-steps-intraday' -> 'dataset') ->> 'value' as steps
    , json_array_elements(data -> 'activities-steps-intraday' -> 'dataset') ->> 'time' as tme
    FROM activity_journals
    WHERE activity_type = 'steps'
      AND archived = FALSE
),
heart_data AS (
    SELECT
    user_id
    , (data -> 'activities-heart' ->> 0)::json ->> 'dateTime' as heart_day
    , json_array_elements(data -> 'activities-heart-intraday' -> 'dataset') ->> 'value' as bpm
    , json_array_elements(data -> 'activities-heart-intraday' -> 'dataset') ->> 'time' as tme
    FROM activity_journals
    WHERE activity_type = 'heart_rate'
      AND archived = FALSE
)
SELECT
  sd.user_id AS user_id
 , sd.steps::INT AS steps
 , hd.bpm::INT as bpm
 , hd.heart_day as journal_date
 , CONCAT(sd.step_day, ' ', sd.tme)::TIMESTAMP AS step_dttm
 , CONCAT(hd.heart_day, ' ', hd.tme)::TIMESTAMP AS heart_dttm
FROM step_data AS sd
    JOIN heart_data AS hd ON hd.heart_day = sd.step_day
        AND hd.tme = sd.tme
        AND hd.user_id = sd.user_id
"""
comb_journals = pd.read_sql(query, conn, parse_dates=['step_dttm', 'heart_dttm', 'journal_date'])
comb_journals

,user_id,steps,bpm,journal_date,step_dttm,heart_dttm
0,1,0,62,2016-04-13,2016-04-13 00:00:00,2016-04-13 00:00:00
1,1,0,61,2016-04-13,2016-04-13 00:01:00,2016-04-13 00:01:00
2,1,0,61,2016-04-13,2016-04-13 00:02:00,2016-04-13 00:02:00
3,1,0,61,2016-04-13,2016-04-13 00:03:00,2016-04-13 00:03:00
4,1,0,63,2016-04-13,2016-04-13 00:04:00,2016-04-13 00:04:00
5,1,0,63,2016-04-13,2016-04-13 00:05:00,2016-04-13 00:05:00
6,1,0,63,2016-04-13,2016-04-13 00:06:00,2016-04-13 00:06:00
7,1,0,63,2016-04-13,2016-04-13 00:07:00,2016-04-13 00:07:00
8,1,0,61,2016-04-13,2016-04-13 00:08:00,2016-04-13 00:08:00
9,1,0,63,2016-04-13,2016-04-13 00:09:00,2016-04-13 00:09:00


In [3]:
# a little verification for sanity check
comb_journals.describe()

,user_id,steps,bpm
count,431878.000000,431878.000000,431878.000000
mean,1.471886,5.359488,63.091262
std,0.499210,17.516386,14.573350
min,1.000000,0.000000,38.000000
25%,1.000000,0.000000,53.000000
50%,1.000000,0.000000,60.000000
75%,2.000000,0.000000,69.000000
max,2.000000,195.000000,205.000000


In [ ]:
comb_journals[['bpm', 'steps']].rolling(5).mean().head()

In [ ]:
comb_journals.groupby('journal_date')[['steps', 'bpm']].sum()

## Explore

In [ ]:
comb_journals[['bpm', 'steps']].quantile([0.9, 0.95, 0.99])

In [ ]:
plt.plot(comb_journals['heart_dttm'], comb_journals['bpm'], 'r.', alpha=0.2)
plt.plot(comb_journals['heart_dttm'], comb_journals['steps'], 'g.', alpha=0.2)
plt.plot(extreme_heart['heart_dttm'], extreme_heart['bpm'], 'b^')
plt.show()

In [ ]:
extreme_heart = comb_journals.copy()
extreme_heart = extreme_heart[ extreme_heart['bpm'] >= extreme_heart['bpm'].quantile(0.95) ]

extreme_heart['beats_step_ratio'] = extreme_heart.apply( lambda eh: eh['bpm'] / (eh['steps'] + 1), axis=1 ) # +1 avoid div by zero problems
extreme_heart.describe()

In [ ]:
high_ratio = extreme_heart[ extreme_heart['beats_step_ratio'] >= extreme_heart['beats_step_ratio'].quantile(0.70) ]

plt.plot(comb_journals['heart_dttm'], comb_journals['bpm'], 'r.', alpha=0.1)
plt.plot(comb_journals['heart_dttm'], comb_journals['steps'], 'g.', alpha=0.1)

plt.plot(high_ratio['heart_dttm'], high_ratio['bpm'], '^')
plt.show()

In [ ]:
yesterday = comb_journals[ (comb_journals['heart_dttm'] < '2017-01-01 10:00:00') &
                           (comb_journals['heart_dttm'] > '2017-01-01 00:00:00')
                         ]

plt.plot(yesterday['heart_dttm'], yesterday['bpm'], 'r.', alpha=0.3)
plt.plot(yesterday['heart_dttm'], yesterday['steps'], 'g.', alpha=0.8)

plt.show()

In [ ]:
reg = comb_journals[ (comb_journals['heart_dttm'] < '2016-12-27 10:00:00') &
                           (comb_journals['heart_dttm'] > '2016-12-27 00:00:00')
                         ]
plt.plot(reg['heart_dttm'], reg['bpm'], 'r.', alpha=0.3)
plt.plot(reg['heart_dttm'], reg['steps'], 'g.', alpha=0.8)

plt.show()

In [ ]:
football = comb_journals[ (comb_journals['heart_dttm'] < '2017-01-01 04:30:59') &
                           (comb_journals['heart_dttm'] > '2016-12-31 14:30:00')
                         ]

plt.plot(football['heart_dttm'], football['bpm'], 'r.', alpha=0.3)
plt.plot(football['heart_dttm'], football['steps'], 'g.', alpha=0.8)

plt.show()

In [ ]:
football['heart_dttm'].iloc[0].time